# Chapter - Problem Decomposition

João Pedro Neto, DI/FCUL

## Introduction

In the context of programming, we are accustomed to solving a complex problem through a sequence of simpler problems. The existence of programming concepts such as functions, classes, objects, or modules were designed to assist the programmer in these kinds of tasks.

<!-- There is also a vast literature on how to perform the analysis and design of problems, introducing techniques that enable the implementation of larger-scale applications. All of this falls under the domain of Software Engineering, a topic beyond the scope of this course. -->

The objective of this chapter is related to this theme, but in a more specific context: given an instance of a problem $P$, how to solve it from simpler instances of $P$. How to decompose a problem into subproblems in order to create an efficient algorithm that solves it?

These questions have a recursive nature. Therefore, it will come as no surprise the importance that recursion will have in this chapter.

We will refer to the following problem-solving techniques:

- Divide and Conquer
- Greedy Algorithms
- Dynamic Programming

These techniques do not have a fixed format or a single algorithm. They refer to ideas and strategies on how to approach and solve problems for which an exhaustive search (what we call «brute force») would result in algorithms with excessive complexity. In each of the sections, we will solve classical problems that, hopefully, help the reader gain intuition about these topics.

## Divide and Conquer

If we didn't have access to the `**` operator, how would we implement the exponentiation operation $x^n$ for $n \geq 0$ as an integer?

In [1]:
def power(x, n):
  res = 1
  for _ in range(n):
    res *= x
  return res

In [5]:
assert power(2,11) == 2048

We used the definition of exponentiation as a sequence of multiplications. This implementation performs a number of iterations proportional to the value of the exponent.

But it's possible to do better. Consider the following equations:

$$x^n = (x^{n/2})^2, \text{n is even}$$

$$x^n = x \times (x^{(n-1)/2})^2, \text{n is odd}$$

This tells us that if we have the power of half of $n$, we can calculate the power of $n$. And this definition can be used recursively to calculate only half of the half, and so on.

The following function implements this idea and performs a number of iterations proportional to the logarithm of the exponent value:

In [4]:
def power(x, n):
  if n==0:
    return 1
  y = power(x, n//2)
  return y*y if n%2==0 else y*y*x

This second approach illustrates the essence of **divide and conquer**.

The divide and conquer technique seeks to divide the original problem into one or more simpler subproblems -- usually half the size --, whose solutions can be used to calculate the solution to the original problem. This problem-solving process has a recursive nature (although it is not mandatory to implement it using recursion).

For example, sorting algorithms like *merge sort* and *quick sort* use this technique when they divide the original sequence into two halves and recursively sort them. Another classic example is *binary search*, which selects only one half of the sequence to continue searching for the desired element.

In general terms, we could consider the following function as a template for this idea:

In [6]:
def divide_conquer(state, divide, conquer, is_base=lambda xs: len(xs)<=1):
  if is_base(state):
    return state

  state1, state2 = divide(state)
  new_state1 = divide_conquer(state1, divide, conquer, is_base)
  new_state2 = divide_conquer(state2, divide, conquer, is_base)
  return conquer(new_state1, new_state2)

For example, *merge sort* could be implemented as follows:

In [7]:
def mergesort(lst):
  def divide(xs):
    return xs[:len(xs)//2], xs[len(xs)//2:]

  def conquer(xs,ys):
    res, i, j = [], 0, 0
    while i<len(xs) and j<len(ys):
      if xs[i] < ys[j]:
        res.append(xs[i]); i += 1
      else:
        res.append(ys[j]); j += 1
    return res + xs[i:] + ys[j:]

  return divide_conquer(lst, divide, conquer)

In [9]:
assert mergesort([7,4,6,1,9,3,8]) == [1, 3, 4, 6, 7, 8, 9]

However, for most problems, it's neither practical nor efficient to rigidly structure the algorithm in this way. We will explore solutions that follow the divide and conquer idea but are more flexible in their implementation.

We will now proceed to showcase several examples of applying this technique.

### The Fibonacci Sequence

For the next problem, let's recall the Fibonacci sequence:

$$F_0 = F_1 = 1$$

$$F_n = F_{n-2} + F_{n-1}$$

A typical function to calculate the n-th value is as follows:

In [10]:
def fib(n):
  x, y = 1, 1
  for _ in range(n-2):
    x, y = y, x+y
  return y

In [11]:
assert fib(60) == 1548008755920

Now, consider the following matrix equivalence:

$$\left[\begin{array}{cc}
0 & 1\\
1 & 1 \end{array} \right]
\left[\begin{array}{c}
x\\
y \end{array}\right]
=
\left[\begin{array}{c}
y\\
x+y \end{array}\right]$$

If we observe the relationship between the values in the two vectors, we notice that this matrix multiplication corresponds to an iteration to calculate the next Fibonacci number!

This tells us that to calculate the n-th number, we need to multiply the matrix $n$ times, starting with an appropriate vector:

$$\left[\begin{array}{cc}
0 & 1\\
1 & 1 \end{array} \right]^n
\left[\begin{array}{c}
1\\
0 \end{array}\right]
=
\left[\begin{array}{c}
F_{n-1}\\
F_{n} \end{array}\right]$$

Matrix exponentiation can be speed-up using the same technique we used for integer exponentiation.

First, we define the multiplication of $2 \times 2$ matrices:

In [12]:
# a 2x2 matrix is represented as a tuple with four components

def multMat(m1, m2):   # multiply  ⌈ a b ⌉ * ⌈ e f ⌉
  a,b, c,d = m1        #           ⌊ c d ⌋   ⌊ g h ⌋
  e,f, g,h = m2
  return a*e+b*g, a*f+b*h, c*e+d*g, c*f+d*h

And the power function has the same structure as the `power` function:

In [13]:
def powerMat(m, n):
  if n==0:
    return 1,0,0,1  # identity matrix

  y = powerMat(m, n//2)
  y_sqr = multMat(y,y)
  return y_sqr if n%2==0 else multMat(y_sqr, m)

With these functions, we can implement the Fibonacci function with a logarithmic number of invocations:

In [14]:
def fib(n):
  return powerMat((0,1,1,1), n)[2]  # return _,_,c,_ = (0,1,1,1)**n

Let's test it with a very large number:

In [15]:
import sys

# changing Python's configuration to handle such a large number!!
sys.set_int_max_str_digits(50_000)

print(f'fib(150k) has {len(str(fib(150_000)))} digits')

fib(150k) has 31348 digits


To conclude, what is the complexity of these two implementations? The immediate answer would be to say that the first one has a complexity of $\mathcal{O}(n)$ and the second one is $\mathcal{O}(\log n)$ in terms of the n-th value of the sequence.

However, there's a detail we're overlooking: these arithmetic operations are not of constant complexity. Adding two numbers with n digits is $\mathcal{O}(n)$, and multiplying is $\mathcal{O}(n \log n)$. This means that their respective complexities will be $\mathcal{O}(n^2)$ and $\mathcal{O}(n (\log n)^2)$.

> note: The complexity referred here is in relation to value $n$. However, value $n$ grows exponentially with the number of bits used - which is the true measure of space. Therefore, when discussing the complexity $C$ of an algorithm in relation to a numeric input $n$, and not its bits, these complexities are called pseudo-$C$. For example, the first Fibonacci function implements a pseudo-quadratic algorithm.

### Karatsuba Multiplication

The Karatsuba algorithm allows for faster multiplication of large integers compared to the traditional multiplication algorithm we learn in school (which has quadratic complexity).

Let's consider integers $x$ and $y$ for which we want to calculate $x \times y$, where the largest number has $n$ digits.

Let $m = 10^{\lfloor n/2 \rfloor}$, then:

$$x = x_1 m + x_0$$

$$y = y_1 m + y_0$$

with $x_0,y_0 < m$.

Developing the multiplication,

$$\begin{array}{lcl}
x \times y & = & (x_1 m + x_0) \times (y_1 m + y_0) \\
           & = & \underbrace{x_1y_1}_{z_2}m^2 + \underbrace{(x_1y_0+x_0y_1)}_{z_1}m + \underbrace{x_0y_0}_{z_0} \\
\end{array}$$

To directly calculate the expressions $z_0, z_1, z_2$ requires four multiplications.

However, the Russian mathematician Anatoly Karatsuba observed that to calculate $z_1$, we don't need two multiplications, but only one (with some extra additions):

$$z_1 = (x_1+x_0)\times(y_1+y_0) - z_2 - z_0$$

In summary, we can perform faster multiplications by breaking numbers into half-digits and recursively calculating the multiplications of simpler numbers. The cost function is given by $T(n) = 3T(n/2) + n$, which [implies](https://www.wolframalpha.com/input?i=T%28n%29+%3D+3T%28n%2F2%29+%2B+n) that the complexity of multiplication decreases from $\mathcal{O}(n^2)$ to $\mathcal{O}(n^{\log_2 3}) \approx \mathcal{O}(n^{1.585})$.

In Python:

In [19]:
def karatsuba(x, y):
  if x < 10 or y < 10: # if one is already quite smaller,
    return x*y         #  just multiply

  n = max(len(str(x)), len(str(y))) # digits of larger number
  m = 10**(n//2)
  x1, x0 = divmod(x, m)
  y1, y0 = divmod(y, m)

  z0 = karatsuba(x0, y0)
  z2 = karatsuba(x1, y1)
  z1 = karatsuba(x1+x0, y1+y0) - z2 - z0

  return z2*m**2 + z1*m + z0

In [20]:
x, y = 123456789, 987654321
assert karatsuba(x, y) == x * y

### Maximum Subarray

The Maximum Subarray problem, also known as the Range Sum Query (RSQ), refers to the problem of finding, in a list of $n$ elements, a sequence of consecutive elements that has maximum sum.

For example, in `23 -10 12 9 -40 32 12 -4 10 6`, the sequence with the maximum sum is `32 12 -4 10 6`, and its total sum is $56$.

A first solution is to test all combinations of sublists and keep the one with the highest sum:

In [21]:
def RSQ(xs):
  n, result = len(xs), 0
  for i in range(n):
    result = max(result, *[sum(xs[i:j+1]) for j in range(i,n)])
  return result

In [22]:
assert RSQ([23,-10,12,9,-40,32,12,-4,10,6]) == 56
assert RSQ([-1,-2,-4,-3]) == 0

This algorithm is $\mathcal{O}(n^2)$.

We can use the divide and conquer technique to achieve a solution with better complexity: we divide the list into two equal-sized lists, calculate the RSQ for both lists, and then combine the results.

First attempt:

In [23]:
def RSQ(xs):
  if len(xs) <= 1:
    return xs[0] if xs else 0
  mid = len(xs)//2
  return max(RSQ(xs[:mid]), RSQ(xs[mid:]))

However, if we test it with the previous assertions, we'll notice that there's a problem with this algorithm: the optimal sequence of elements can be split between the two halves. In other words, we need to be more careful in combining the two results.

We need to include a third value in the result combination. This value includes the middle element and extends the sum to the left and right as far as possible. As long as we can perform this task in linear time, we're good, because the recurrence relation of this recursive function will be

$$T(n) = 2T(n/2) + \mathcal{O}(n) \propto \mathcal{O}(n \log n)$$

an improvement over the previous quadratic solution.

Here's the algorithm in Python:

In [24]:
# this code does not use slicing to reduce the number of list objects
def RSQ(xs, left=None, right=None):
  if left is None:
    left, right = 0, len(xs)-1
  if right-left <= 1:
    return xs[left] if left==right else 0

  mid = (left+right)//2
  maxLsum = currentSum = 0          # compute maximum sum to the left
  for i in range(mid, left-1, -1):
    currentSum += xs[i]
    maxLsum = max(maxLsum, currentSum)

  maxRsum = currentSum = 0          # compute maximum sum to the right
  for i in range(mid+1, right+1):
    currentSum += xs[i]
    maxRsum = max(maxRsum, currentSum)

  return max(maxLsum+maxRsum,       # select highest value between three options
             RSQ(xs, left,  mid),
             RSQ(xs, mid+1, right))

### Number of Inversions

Given a sequence with comparable elements, how far is it from being sorted?

This question is useful when comparing rankings. For example, in a streaming service, you can compare movie ratings among various users (each customer has their own ranking) to find customers with similar tastes. This information is important for making personalized recommendations.

One measure to answer this question is to count how many inversions exist in the sequence, i.e., how many pairs of numbers are in the wrong order. For example, the sequence `[1,3,5,2,4,6]` has three inversions: `(3,2), (5,2), (5,4)`.

An algorithmic approach would be to compare all possible pairs among the $n$ numbers, which would have a time complexity of $\mathcal{O}(n^2)$.

However, applying the divide and conquer technique allows us to improve this complexity. The idea is as follows:

1. Split the sequence into two halves.
2. Calculate the inversions in each half.
3. Calculate the inversions between the two halves.
4. The final result will be the sum of the results from the previous two steps.

This divide and conquer approach can significantly reduce the time complexity when compared to the brute-force method.

How to implement this algorithm efficiently? Each half should be sorted to facilitate the process. Therefore, we have a `merge` function that takes two sorted sequences (very similar to _merge sort_) but with the additional concern of counting how many pairs of values are found inverted (which can be done in linear time). The complexity of this algorithm is $\mathcal{O}(n \log n)$.

For example, consider the list `[3,1,5,2,6,4]`. Each half we recursively invoke, `[3,1,5]` and `[2,6,4]`, will return as sorted `[1,3,5]` (with one inversion detected) and `[2,4,6]` (also with one inversion). The merging process will detect that `3` is inverted with `2`, and that `5` has two inversions with `2` and `4`. Thus, the total number of inversions is 1 + 1 + 3 = 5.

In Python:

In [25]:
def n_inversions(xs, a=None, b=None):
  if a is None:
    a, b = 0, len(xs)
  if a == b-1:
    return 0
  # split sequence into two halves
  mid = (a+b)//2
  return n_inversions(xs, a, mid) + n_inversions(xs, mid, b) + merge(xs, a, mid, b)

def merge(xs, a, mid, b):
  count, tmp = 0, [0]*(b-a)
  low, high = a, mid
  for i in range(a, b):  # merge sequences
    if high >= b or (low < mid and xs[low] <= xs[high]):
      tmp[i-a] = xs[low]
      low += 1
    else:
      count += mid-low # count number of inversions found
      tmp[i-a] = xs[high]
      high += 1
  xs[a:b] = tmp
  return count

In [26]:
assert n_inversions([3,1,5,2,6,4]) == 5
assert n_inversions([1,3,5,2,4,6]) == 3

### Nearest Neighbors

Another classic problem is finding, given $n$ points, the pair of points that are closest to each other.

The brute-force approach tells us to compare the distance between all pairs of points, which results in an $\mathcal{O}(n^2)$ algorithm.

In [27]:
def dist(p, q):
  return ((p[0]-q[0])**2 + (p[1]-q[1])**2)**0.5

def closest_bruteforce(ps):
  return min((dist(p,q),p,q) for i,p in enumerate(ps)
                             for   q in ps[i+1:])

But we can implement a smarter solution:

1. Sort the points by the x-coordinate.

2. Divide the points into two sets separated by the median line of the points.

3. Invoke the algorithm recursively for these two halves, obtaining the minimum distances $d_L$ and $d_R$.

4. Find the minimum distance $d_{LR}$ between points from the two halves.

5. Return min($d_L$, $d_R$, $d_{LR}$).


In step 4, for each point near the boundary -- points within a distance less than min($d_L$, $d_R$) --, it can [be](https://www.cs.ubc.ca/~liorma/cpsc320/files/closest-points.pdf) [shown](https://github.com/jpneto/Prog.I/raw/master/docs/closest-points.pdf) that there are at most six points for which you need to calculate the distance.

This implies that step 4 can be performed in linear time (see the `closest_border` function), and the overall complexity of the algorithm is $\mathcal{O}(n \log n)$.

The main function:

In [28]:
def closest(ps, xx=None, yy=None):
  if xx is None:
    # this sorting is useful to speed up some parts of the algorithm
    xx = sorted(ps, key=lambda p:p[0]) # sort by x coordinates
    yy = sorted(ps, key=lambda p:p[1]) # sort by y coordinates

  if len(xx) < 6: # with a small number of pts, better to brute force
    return closest_bruteforce(xx)

  # split points into two halves
  mid = len(xx)//2
  leftX,  leftY  = xx[:mid], [pt for pt in xx if pt[0] <= xx[mid][0]]
  rightX, rightY = xx[mid:], [pt for pt in xx if pt[0] >  xx[mid][0]]

  # search closest points for each half
  dL, pL, qL = closest(ps, leftX,  leftY)
  dR, pR, qR = closest(ps, rightX, rightY)

  if dL <= dR:            # select minimum half
    d, p, q = dL, pL, qL
  else:
    d, p, q = dR, pR, qR

  dLR, pLR, qLR = closest_border(xx, yy, d, p, q) # search border

  if d <= dLR:            # compare minimum half with border result
    return d, p, q
  else:
    return dLR, pLR, qLR

and the function that calculates the minimum distance between points on the boundary:

In [29]:
def closest_border(xx, yy, d, p, q):
  midX = xx[len(xx)//2][0]
  pts   = [x for x in yy if midX-d <= x[0] <= midX+d] # points not further than d from mid_x

  dLR, pLR, qLR = d, p, q
  for i in range(len(pts)-1):
    # check only 6 pts after each pts[i]; cf. [Cormen 3ed, pgs 1039-43]
    for j in range(i+1, min(i+7, len(pts))):
      d2 = dist(pts[i], pts[j])
      if d2 < dLR:
        dLR, pLR, qLR = d2, pts[i], pts[j]
  return dLR, pLR, qLR

A use case with the random generation of many points to compare the results obtained by the two solutions:

In [30]:
from random import randint, seed

def rnd_pts(n, M=10**4, seed_id=124):
  seed(seed_id)
  return [(randint(-M, M), randint(-M, M)) for i in range(n)]

ps = rnd_pts(2000)
assert closest_bruteforce(ps) == closest(ps)

%timeit -n1 -r5 closest_bruteforce(ps)
%timeit -n1 -r5 closest(ps)

815 ms ± 1.76 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
14.9 ms ± 170 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


### Meet in the Middle

Consider the following problem: Given a list $xs$ with at most $40$ elements, output how many subsets of $xs$ sum to $x$.

Let's first try a brute force solution: just compute the powerset of $xs$ and count those that sum to $x$.

In [31]:
from itertools import combinations

def powerset(xs):
  yield from (s for n in range(len(xs)+1)
                for s in combinations(xs, n))

def subset_sum(xs, x):
  return sum(sum(subset) == x for subset in powerset(xs))

assert subset_sum(range(6), 5) == 6 # [5], [0,5], [1,4], [2,3], [0,1,4], [0,2,3]
assert subset_sum(range(12), 18) == 64

Computing the powerset is $\mathcal{O}(2^n)$ so this means around $2^{40}$ operations as a worst case, i.e., a trillion operations, which is just too large.

Now consider a different approach: split the original list into two equal size sublists. Compute the powerset of each. Then, for a left result of $y$ check if exists a right result of $y-x$. Every time this happens accumulate the counter.

In [32]:
from collections import Counter

def subset_sum2(xs, x):
  mid = len(xs)//2
  left_sums  = Counter(sum(s) for s in powerset(xs[:mid]))
  right_sums = Counter(sum(s) for s in powerset(xs[mid:]))

  return sum(left_sums[y] * right_sums[x-y] for y in range(x+1))

assert subset_sum2(range(6), 5) == 6
assert subset_sum2(range(12), 18) == 64

By splitting the problem in half, and searching compatible values from both partial solutions, we get an algorithm $\mathcal{O}(n \cdot 2^{n/2})$, still exponential but with a slower growth.

For $n=40$ the previous trillion number of operations reduces to tens of millions of operations.

In [33]:
%timeit subset_sum(range(21), 23)
%timeit subset_sum2(range(21), 23)

448 ms ± 5.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
686 µs ± 3.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


This in essence is the strategy denoted **meet in the middle** or **bidirectional search**. It works on difficult problems when the instance size is too large for a brute force approach, but not that large to brute force half the input.

A context where this technique is useful is when searching a path thru a very large graph. It will be faster to start from both directions and try to find a common node.

It is possible to process this using a search technique, as BFS, by seeding the two starting points, and processing each path alternatively. By keeping which nodes were visited in a cache, if a node from one path is in the cache from the other path, then we found the common node.

## Greedy Algorithms

In a problem with **optimal substructure**, it is possible to find the optimal solution by combining the optimal solutions of its subproblems.

In these problems, if it can be demonstrated that at each step we can choose only the «best» local option to find the global optimal solution (referred to as the _**greedy choice property**_), then we can use the so-called **greedy algorithms**.

> Proving when a problem is amenable to a greedy solution requires non-trivial mathematical arguments. For this reason, we will not delve into the [theory](https://en.wikipedia.org/wiki/Matroid) used, as it is beyond the scope of this course.

A greedy algorithm, by selecting only one subproblem to compute, tends to be very fast.

It is not hard to find one or more greedy strategies for a given problem. Unfortunately, a greedy approach does not always work to find the optimal solution, even if it often provides an approximate solution that is reasonably close to that optimal solution (this will be the topic of a future chapter).

A classic example is minimizing the number of coins needed to represent an amount. For instance, given the Euro coins, what is the minimum number of coins to sum up to $n$ cents?

A greedy algorithm that, for the current amount, chooses the largest possible coin allows us to find the answer:

In [34]:
def minCoins(n, coins=[200,100,50,20,10,5,2,1]): # coins in cents, reversed sorted
  result = 0
  for coin in coins:
    result += n//coin # take as many as possible from the current coin
    n %= coin         # just take care of the remainder
  return result

In [35]:
assert minCoins(458) == 6   # 200+200+50+5+2+1 -> six coins

It's worth noting that this reasoning of choosing the largest coin at each step doesn't work for all possible sets of coins$^{(*)}$. For example,

In [36]:
minCoins(6, coins=[4,3,3,1]) # best solution is 3+3 -> two coins

3

> <font size="-1">$^{(*)}$ When the greedy approach works for the coin problem, it is said that the set of coins is _canonical_, but determining whether a set of coins is canonical or not is a non-trivial task, [[1](https://math.stackexchange.com/questions/4139845/change-making-problem-pearson-algorithm-to-check-the-optimality-of-greedy-solu), [2](https://math.stackexchange.com/questions/3121896/what-property-of-a-coin-system-makes-it-canonical)].</font>

Two mentioned problems in the chapter about Graphs, finding the shortest path between two nodes (via Dijkstra's algorithm) and finding the minimal spanning tree, can both be solved using greedy algorithms. The $A^*$ search algorithm also employs a greedy strategy to choose the next candidate for exploration.

Let's explore some other classic problems that are solved using this technique.

### Knapsack Problem (fractional version)

The knapsack problem corresponds to having a set of objects, each with a weight $w_i$ and a value $v_i$, that we can place in a backpack with a certain capacity $C$. We wish the maximize the value of what we can carry in the backpack.

The fractional version tells us that we can divide the objects into units of weight (e.g., kilograms) before putting them in the backpack.

This problem has a greedy solution: we should sort the objects by their values per kilogram and fill the knapsack with the most valuable kilograms first.

In [37]:
def fracKnapsack(capacity, values, weigths):
  units = [v/weigths[i] for i,v in enumerate(values) for _ in range(weigths[i])]
  units.sort(reverse=True)
  return sum(units[:capacity])

In [38]:
fracKnapsack(7, values=[4,8,2], weigths=[8,4,5])

9.5

### Task Scheduling

Let's consider a set of tasks with start and end times that can be scheduled in a room. There are overlapping time slots, which means we have to decide which tasks to schedule. We want to book the room for the maximum number of tasks possible.

This problem can be solved with a greedy algorithm: among the available tasks, schedule the one that finishes as early as possible and is compatible with the tasks already scheduled.

Any compatible task that finishes later cannot result in a better solution. For example, if I have two compatible tasks, one ending at 15:00 and the other ending at 16:00, it is never preferable to choose the one ending at 16:00.

The following function implements this idea. To facilitate the algorithm, it sorts the list of tasks in ascending order according to their end time.

In [39]:
def schedule(tasks):
  tasks.sort(key=lambda t: t[1])   # sort by ending time
  result = [tasks[0]]              # always add first task
  k = 0
  for m in range(1,len(tasks)):
    if tasks[m][0] >= tasks[k][1]: # select the compatible task that ends sooner
      result.append(tasks[m])
      k = m
  return result

In [40]:
tasks = [(1,4), (6,10), (3,5), (0,6), (5,9), (5,7), (2,14), (12,16), (3,9), (7,11), (8,12)]
schedule(tasks)

[(1, 4), (5, 7), (7, 11), (12, 16)]

An alternative solution would be to choose the task that starts as late as possible while being compatible with the already scheduled tasks. With this option, the list of tasks would be constructed from the end to the beginning.

### Interval Coverage

The interval covering problem is as follows:

Given an initial interval $[L, R]$ and a set of intervals $[a_i, b_i]$, find the minimum number of these intervals needed to cover the entire initial interval. Interval intersections are allowed.

This problem also has a greedy solution. First, we sort the intervals in ascending order of their start values $a_i$, and in case of ties, we sort in descending order of their end values $b_i$. For example, $[0,2] < [0,1] < [1,2]$.

The idea is to keep track of the best interval encountered so far that satisfies having its start value $a_i$ to the left of the desired value. When an interval appears with an end value $b_i$ greater than the current best, we update the best interval since it is better than the intervals found so far. For example, if $L=0$ and we have the interval $[-1,4]$, and then the interval $[0,5]$ appears, we prefer to keep the latter because it covers the initial interval starting from zero.

When an interval appears with a start value $a_i$ to the right of the desired value, we need to keep the best interval found so far. Continuing the example, if after $[0,5]$, we encounter $[2,7]$, we must keep $[0,5]$ because no more intervals will appear that will cover zero (remember that we sorted the intervals).

The new interval may or may not be interesting. In the case of $[2,7]$, we keep it (for now). Say, the next interval is $[2,4]$. This is not useful because those values are already included in $[0,5]$ that we just kept. If the next interval is $[5,10]$ we can keep it (for now) and drop $[2,7]$.

We repeat this process until the last interval kept has an end value $b_i$ greater than $R$, or a situation occurs where one of the values we are looking for to the left or right cannot be satisfied. In the second case, we need to report that the problem instance has no solution.

The following function implements the described algorithm:

In [41]:
from collections import namedtuple
Interval = namedtuple('Interval', 'left right')

def coverage(L, R, intervals):
  intervals.sort(key=lambda p:(p[0],-p[1]))
  result = []
  needLeft = L                      # from where we still need to cover
  currentBest = Interval(L,L)       # our best interval so far
  haveCurrentBest = False
  feasible = True

  for a,b in intervals:
    if a <= needLeft:               # is this an interval that covers needLeft?
      if b > currentBest.right:     # and covers even more to the right?
        currentBest = Interval(a,b) # if so, keep it
        haveCurrentBest = True
    else:                           # ok, no more intervals will cover needLeft
      if haveCurrentBest:           # if we have a good interval
        result.append(currentBest)  #  include in it the solution
        needLeft = currentBest.right
        haveCurrentBest = False
        if currentBest.right >= R:  # no need to continue, found a minimal cover
          break
      if a <= needLeft:             # needLeft might've changed, need to recheck
        if b > currentBest.left:
          currentBest = Interval(a,b)
          haveCurrentBest = True
      else:                         # if this interval does not cover needLeft
        feasible = False            # no next ones will cover it,
        break                       # so no solution exists

  if haveCurrentBest:                  # don't forget the last interval
    result.append(currentBest)
    feasible = currentBest.right >= R  # but it might not be enough...

  return [(a,b) for a,b in result] if feasible else None

A use case:

In [42]:
intervals = [(34,59), (-16,-14), (1,4), (30,52), (46,66), (3,14), (46,52), (15,31), (38,55),
             (46,49), (14,17), (69,85), (16,37), (54,66), (-7,0), (-15,5), (37,39), (-14,-13)]
coverage(-12, 33, intervals)

[(-15, 5), (3, 14), (14, 17), (16, 37)]

### Huffman Coding

This problem is from the field of Information Theory, regarding information compression.

The common way to represent characters in binary is through the ASCII encoding (now part of the Unicode standard) that dedicates a fixed number of seven bits to each letter. We can observe some examples with the help of the following functions:

In [43]:
to_binary   = lambda c: bin(ord(c))[2:] # convert to ASCII binary representation
binary_code = lambda msg: ''.join(map(to_binary, msg)) # convert string to bits

In [44]:
print(binary_code('h'))
print(binary_code('hello world'))

1101000
1101000110010111011001101100110111110000011101111101111111001011011001100100


However, this encoding is not efficient when it comes to the number of bits used. We know that some letters are more frequent than others, yet all letters consume the same amount of memory. ASCII was designed to facilitate computer memory management. However, for data transmission, it's not an efficient option.

The issue of reducing the number of bits for communication purposes was a concern as far back as the telegraph era. Morse code reflects the frequencies of letters in the English language: an `e`, the most common letter, is represented by `.`; while the letter `q` is represented by `--.-`, a string four times longer.

In 1951, David Huffman found an algorithm that, given the frequencies of the letters in question, can calculate an optimal encoding in terms of memory usage. The algorithm has a greedy nature and is highly effective at finding the answer:

1. Create an empty min-priority queue.

2. For each unique letter in the message, create a binary tree with that letter and its frequency and insert it into the queue.

3. While the queue has more than one tree:

   a. Remove the two trees with the lowest priority (priority is determined by the tree's frequency).

   b. Create a new tree with a frequency equal to the sum of the frequencies of the two removed trees, and make the two removed trees its children.

   c. Insert this new tree back into the queue.

For example, for the letters `"abcdefghi"` with frequencies `[4, 5, 6, 9, 11, 12, 15, 16, 20]`, the algorithm should create the following tree:

<center><img src='https://raw.githubusercontent.com/jpneto/Prog.I/master/imgs/huffman_tree.png' width=650px><br>ref: Magnus Hetland, «Python Algorithms»</center>

Function `huffman_tree` implements this algorithm:

In [46]:
from heapq     import heapify, heappush, heappop
from itertools import count

def huffman_tree(seq, freq):
  """ ref: Magnus Hetland, 'Python Algorithms', chapter 7 """
  num = count()
  trees = list(zip(freq, num, seq))             # «num» ensures a valid ordering
  heapify(trees)                                # min-heap based on frequencies
  while len(trees) > 1:                         # until there's just one tree:
    fa, _, a = heappop(trees)                   #  get the two 'smallest' trees
    fb, _, b = heappop(trees)
    heappush(trees, (fa+fb, next(num), [a, b])) #  combine and add new tree
  return trees[0][-1]

Let's compute the coding tree for the previous example:

In [47]:
seq = "abcdefghi"
frq = [4, 5, 6, 9, 11, 12, 15, 16, 20]
huffman_tree(seq, frq)

[['i', [['a', 'b'], 'e']], [['f', 'g'], [['c', 'd'], 'h']]]

We can use function `codes` to calculate the codes for each letter:

In [50]:
def codes(tree, prefix=""):
  """ ref: Magnus Hetland, 'Python Algorithms', chapter 7 """
  if len(tree) == 1:
    yield (tree, prefix)                      # a leaf with its code
  else:
    for bit, child in zip("01", tree):        # left is 0, right is 1
      yield from codes(child, prefix + bit)   # get codes recursively

In [51]:
list(codes(huffman_tree(seq, frq)))

[('i', '00'),
 ('a', '0100'),
 ('b', '0101'),
 ('e', '011'),
 ('f', '100'),
 ('g', '101'),
 ('c', '1100'),
 ('d', '1101'),
 ('h', '111')]

Let's see the result it produces with a slightly longer sentence.

Here is the ASCII binary representation:

In [52]:
msg = 'an example sentence with only lowercase letters that can be used to test the algorithm we are explaining'
msg_ascii = binary_code(msg)
msg_ascii

'110000111011101000001100101111100011000011101101111000011011001100101100000111001111001011101110111010011001011101110110001111001011000001110111110100111101001101000100000110111111011101101100111100110000011011001101111111011111001011110010110001111000011110011110010110000011011001100101111010011101001100101111001011100111000001110100110100011000011110100100000110001111000011101110100000110001011001011000001110101111001111001011100100100000111010011011111000001110100110010111100111110100100000111010011010001100101100000110000111011001100111110111111100101101001111010011010001101101100000111011111001011000001100001111001011001011000001100101111100011100001101100110000111010011101110110100111011101100111'

To create the Huffman code for any string, we define two more auxiliary functions:

In [53]:
from collections import Counter

def encoding_map(msg):
  """ return unique letters from 'msg' and their frequencies """
  data = Counter(msg).items()
  seq, freq = ''.join(map(lambda p:p[0], data)), list(map(lambda p:p[1], data))
  return {letter:code for letter, code in codes(huffman_tree(seq, freq))}

def huffman_encode(msg, encoding):
  """ apply Huffman encoding to msg """
  return ''.join(map(lambda c: encoding[c], msg))

Then:

In [54]:
encoding = encoding_map(msg)
msg_huffman = huffman_encode(msg, encoding)
msg_huffman

'10011000111101000001001000010001001101011110011101100001010110000111010111101111110000101100111111010100001100010001110110110100111110111011011101001001110111101101010100101011101100111110101100110010101110111010011000111001001101111001010001110100101111101011010111010101001101011101011001101111100101100001111010110111100001011001000011110111110111110011101110111110100000000100110100111000100011000100000011'

We can already see that the encoded string has become shorter. Let's calculate the compression ratio:

In [55]:
print(f'{len(msg_huffman) / len(msg_ascii):3.1%}')

57.7%


We have reduced the binary representation by almost half! Of course, to decode it, we need to transmit the binary tree with the codes, which may not be efficient for transmitting a small number of characters. However, for texts with hundreds or more characters, compression has a clear advantage.

To complete the example, let's perform the decoding:

In [56]:
def decoding_map(encoding):
  """ just invert encoding dictionary """
  return {v:k for k,v in encoding.items()}

def huffman_decode(bits, decoding):
  """ apply decoding to binary message 'bits' """
  key, res = '', []
  for bit in bits:
    key += bit
    if key in decoding:
      res.append(decoding[key])
      key = ''
  return ''.join(res)

In [57]:
huffman_decode(msg_huffman, decoding_map(encoding))

'an example sentence with only lowercase letters that can be used to test the algorithm we are explaining'

A final quote, not about greedy algorithms, but about free lunches:

> If a lossless compression reduces the size of one input, the pigeonhole principle says it must increase the size of some input. That's usually not a problem because in practice you care more about *probable* inputs than *possible* inputs." — John D Cook

(imagine a telegrapher sending a sequence of one thousand `q`'s...)

## Dynamic Programming

Dynamic programming (DP) is a problem-solving technique for problems with optimal substructure, much like greedy algorithms. However, DP has a broader application domain and, when applicable, ensures the correctness of the solution.

The characteristic that problems must have, for DP to be used, is that they exhibit _overlapping subproblems_, meaning that subproblems share common sub-subproblems. When this occurs, a divide-and-conquer solution will inevitably perform more computation than necessary. What a DP algorithm does is store the solutions of the subproblems it solves, so that they can be reused later. This allows for the creation of algorithms with better time complexity at the cost of some increase in space complexity.

One difference from the divide-and-conquer technique is that DP decomposes the problem into overlapping subproblems or partitions that are not mutually exclusive, while divide-and-conquer algorithms work better when such overlap is not present.

The primary areas where DP is applied are in counting and optimization. In the rest of this section, we will solve some classical examples to help you understand how to use this technique for analyzing and solving problems.

As a first example, let's consider again the problem of returning the smallest number of coins.

Given a total of $n$ cents, with coins $c_1, c_2, \ldots, c_k$, and $M_n$ being the smallest number of coins that sum to $n$ cents, we can decompose the problem using the following recursive relationship:

$$M_0 = 0$$

$$M_n = 1 + \min(M_{n-c}), ~ ~ c \in \{c_1, c_2, \ldots, c_k\}$$

In other words, we need zero coins to make change for zero cents (the base case), and to make change for $n$ cents, we have to choose the coin $c$ that minimizes the number of coins for the remaining change, i.e., $n-c$ cents.

In Python:

In [58]:
from math import inf as oo

def minCoins(n, coins, cache=None):
  if cache is None:
    cache = {0:0} # zero cents need zero coins

  if n not in cache:
    cache[n] = 1 + min((minCoins(n-c, coins, cache)
                        for c in coins if c<=n), default=+oo)
  return cache[n] # the default value deals with empty (impossible) combinations

In [63]:
assert minCoins(458, (200,100,50,20,10,5,2,1)) == 6
assert minCoins(6, (4,3,3,1)) == 2              # this instance failed in the greedy approach
assert minCoins(6249, (186,419,83,408)) == 20
assert minCoins(62, (186,419,83,408)) == +oo    # impossible instance

This solution is an example of a **top-down** approach. The algorithm is recursive, where the current problem instance invokes simpler instances, using memoization to avoid redundant computations. The complexity is $\mathcal{O}(n)$ because we calculate at most $n+1$ distinct values (the change from zero to $n$ cents).

> As a sidenote, Python provides a caching decorator `cache` that performs automatic memoization. Solving this problem again:

In [64]:
from functools import cache

@cache
def minCoins(n, coins):
  if n==0:
    return 0 # zero cents need zero coins

  return 1 + min((minCoins(n-c, coins) for c in coins if c<=n), default=+oo)

Certain problem resolutions work better in a **bottom-up** approach. In these solutions, we start with the smallest problem instances and build larger instances from them. Typically, these solutions result in iterative algorithms rather than recursive ones.

Let's look at an implementation of the same problem using this approach.

The following algorithm creates a list `sols` to store solutions. Each index `i` of the list represents the minimum number of coins to make change for `i` cents.

The loop will iterate through all the intermediate amounts, calculating the minimum based on previous positions in the list. For example, if we have coins of $10$ and $25$ cents, to calculate `sols[30]`, we will use the values of `sols[20]` ($30-10$ cents) and `sols[5]` ($30-25$ cents).

In [65]:
def minCoins(n, coins):
  sols = [0] + [inf]*n  # sols[i] == minimum coins for amount i

  for i in range(1,n+1):
    sols[i] = 1 + min((sols[i-c] for c in coins if c<=i), default=+oo)

  return sols[n]

Complexity is again $\mathcal{O}(n)$.

Optimization problems are often associated with similar counting problems. For the problem of finding the minimum number of coins, we can solve a similar problem of calculating how many different combinations of coins, denoted as $C_n$, sum to $n$ cents.

For example, with coins of $1$, $3$, and $4$ cents, there are six ways to make $5$ cents: `1+1+1+1+1`, `1+1+3`, `1+3+1`, `1+4`, `3+1+1`, `4+1` (here, the order of the coins matters).

In this case, the problem decomposition would be as follows:

$$C_0 = 1$$

$$C_n = \sum_{c ~ \in ~ \text{coins}} C_{n-c}$$

There's one way to represent zero cents (with no coins). For any other amount, we take each of the coins one by one, calculate these subproblems, and return their sum.

In Python:

In [66]:
@cache
def countCoins(n, coins):
  if n==0:
    return 1  # there's one way to represent zero cents

  return sum(countCoins(n-c, coins) for c in coins if c<=n)

In [67]:
assert countCoins(5, (1,3,4)) == 6

# how many ways are there to sum one Euro?
assert countCoins(100, (1,2,5,10,20,50,100,200)) == 119310010658418457688202

It is not easy to formulate a dynamic programming solution. One approach is to think about the structure of an optimal solution, and how it can be derived from subproblems. This tends to recursive algorihtms that can be, if the programmer prefers, adapted to an iterative, bottom-up approach that fills arrays or matrices with subsolutions.

### Cutting Sticks

Consider that you have a tree trunk of length $n$ meters and a cutter that costs $m$ credits to cut a trunk of $m$ meters. You want to cut the trunk at positions $m_1, m_2, \ldots, m_k$ while minimizing the total number of credits spent.

Let's consider an instance with a trunk of length $n=10$ meters and the cuts at $2, 4, 7$ meters. There are multiple possible solutions. For example, if the sequence of cuts is $2, 7, 4$, the cost will be $25$ credits. If it is $7, 4, 2$, the cost will be $21$ credits.

<center><img src='https://raw.githubusercontent.com/jpneto/Prog.I/master/imgs/cutting_sticks.png' width=500px></center>

This problem exhibits optimal substructure: the optimal cut of a trunk section is always the same and can be used to cut trunks that include that section.

Furthermore, the subproblem of cutting the section from the first, say, 75% of the trunk shares sub-subproblems with the subproblem of cutting the last 75% of the trunk. In other words, there is an overlap of subproblems. These two characteristics indicate that we can solve the original problem using dynamic programming.

Let $C_{i, j}$ be the minimum cost of cutting between measurements $i$ and $j$. The recursive relationship that defines this problem is as follows:

$$C_{i,i} = C_{i,i+1} = 0$$

$$C_{i,j} = \min(C_{i,k} + C_{k,j} + (m_j-m_i) ), ~ ~ k \in \{i+1, \ldots, j-1\}$$

The following **top-down** solution recursively tests the cost of cutting each of the options $m_k$ and stores the one that results in the minimum value. To avoid an exponential search for solutions, we store the previously found solutions in a cache:

In [68]:
def cutSticks(n, cuts, i=None, j=None, cache=None):
  if cache is None:
    cuts = [0]+cuts+[n]               # add extremities
    i, j = 0, len(cuts)-1             # define search interval
    cache = {}                        # init cache
    for k in range(n+1):
      cache[k,k] = cache[k,k+1] = 0   # sticks of size 0 and 1 have no cost

  if (i,j) not in cache:
    cache[i,j] = min(cutSticks(n, cuts, i, k, cache) +
                     cutSticks(n, cuts, k, j, cache) + cuts[j] - cuts[i]
                     for k in range(i+1,j))
  return cache[i,j]

The solution has to go through all combinations of $m$ cuts, resulting in a time complexity of $\mathcal{O}(m^2)$.

Here are some use cases:

In [69]:
assert cutSticks( 10, [2,4,7]) == 20
assert cutSticks(100, [25,50,75]) == 200
assert cutSticks(725, [34,38,90,92,136,168,314,386,501,529,560,661]) == 2486

### Longest Increasing Subsequence

The Longest Increasing Subsequence (LIS) problem is the task of, given a list of elements, finding the longest sequence of increasing elements, not necessarily adjacent to each other.

For example, for the list `[4,2,8,5,6,3]`, two possible LIS are `[4,5,6]` or `[2,5,6]`. We cannot include the eight because it would prevent us from including the five and the six.

Our bottom-up solution starts by calculating the increasing subsequences from index zero up to index `i`. With this information, we can then calculate the increasing subsequence for index `i+1`.

In the example list `[4,2,8,5,6,3]`, our progressive calculations would create the following increasing subsequences:

    0  [4]
    1  [2]
    2  [4,8]
    3  [4,5]
    4  [4,5,6]
    5  [2,3]

The LIS will be the increasing subsequence with the greatest length.

In [70]:
def LIS(xs):
  incs = [[] for _ in range(len(xs))] # incs[i] will have the IS from xs[:i]
  incs[0].append(xs[0])               # initialize with the first IS

  for i in range(1,len(xs)):
    js = [j for j in range(i) if xs[j] < xs[i]]              # select all possible candidates
    if js:
      incs[i] = incs[max(js, key=lambda j: len(incs[j]))][:] # clone the largest IS to incs[i]
    incs[i].append(xs[i])                                    # and add the current xs[i]

  return max(incs, key=len) # LIS is the largest IS (duh!)

Due to the loop and list comprehension, the algorithm is $\mathcal{O}(n^2)$.

A use case:

In [71]:
xs = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15, 3]
LIS(xs)

[0, 4, 6, 9, 13, 15]

> It is possible to introduce optimizations to solve the problem in $\mathcal{O}(n \log k)$, where $k$ is the dimension of the solution. We will not discuss the [algorithm](https://en.wikipedia.org/wiki/Longest_increasing_subsequence).

### Knapsack Problem (integer version)

Let's go back to the knapsack problem described above with a modification: it's necessary to choose the whole object (we cannot split it into units). With this new rule, a greedy solution -- starting by choosing the fractional units from most valuable object -- no longer works.

But we can use DP to solve the problem. Let $C$ be the capacity, and $k$ objects with values $v_1, \ldots, v_k$ and weights $w_i, \ldots, w_k$.

The total value $V_{C,S}$ for capacity $C$ and set $S$ of available objects is given by:

$$V_{0,S} = 0$$

$$V_{C,\emptyset} = 0$$

$$V_{C,S} = \max( v_1 +V_{C-w_1,S-\{1\}} , V_{C,S-\{1\}} )$$

In summary, the value is zero if the capacity is zero or there are no more objects to choose from (the base cases of recursion). Otherwise, the value is the maximum of two options: choosing the first object (if it's not too heavy) or not choosing.

This is where the recursion step comes in. We have again a top-down approach, and we must remember to memorize the solutions we find:

In [72]:
def knapsack(capacity, values, weigths, idx=0, cache=None):
  if cache is None:
    cache = {}

  key = (idx,capacity)
  if key not in cache:
    if idx==len(values) or capacity==0:
      cache[key] = 0
    elif weigths[idx] > capacity: # if item is too heavy, skip it
      cache[key] = knapsack(capacity, values, weigths, idx+1)
    else:
      cache[key] = max(values[idx] +
                       knapsack(capacity-weigths[idx], # either use it,
                                values, weigths, idx+1, cache),
                       knapsack(capacity,              # or not
                                values, weigths, idx+1, cache))
  return cache[key]

Some use cases:

In [73]:
assert knapsack(10, [10,19,10], [5,6,5]) == 20
assert knapsack(24, [13,30,15,12,36,5,12], [5,10,6,2,19,1,2]) == 75

In this implementation, we are only returning the maximum value, not the objects that should be chosen. To do that, we would need to adapt the algorithm to also store the chosen elements in the cache.

In [74]:
def knapsack2(capacity, values, weigths, idx=0, cache=None):
  if cache is None:
    cache = {}

  key = (idx,capacity)
  if key not in cache:
    if idx==len(values) or capacity==0:
      cache[key] = (0, [])
    elif weigths[idx] > capacity:  # if item is too heavy, skip it
      cache[key] = knapsack2(capacity, values, weigths, idx+1)
    else:
      sol1 = knapsack2(capacity-weigths[idx], values, weigths, idx+1, cache)
      sol2 = knapsack2(capacity,              values, weigths, idx+1, cache)
      if sol1[0]+values[idx] >= sol2[0]:
        cache[key] = (sol1[0]+values[idx], sol1[1]+[idx])
      else:
        cache[key] = (sol2[0]            , sol2[1]+[idx])
  return cache[key]

Note that now, in addition to the maximum value, the indices of the chosen objects are also returned:

In [75]:
knapsack2(24, [13,30,15,12,36,5,12], [5,10,6,2,19,1,2])

(75, [5, 3, 2, 1, 0])

### Matrix Multiplication


Consider that we need to multiply $n$ matrices in sequence, $A_1 A_2 \ldots A_n$. It is known that the dimensions between consecutive matrices must be compatible. However, the order in which we choose to multiply them has consequences for the number of arithmetic operations required.

For example, consider three matrices $A_1, A_2, A_3$ with dimensions $10\times 50, 50 \times 5, 5 \times 100$. There are two multiplication options:

- If we choose $(A_1 A_2) A_3$, we need $10\times 50 \times 5 + 10 \times 5 \times 100 = 7500$ operations.
- If we choose $A_1(A_2A_3)$, we will spend $50\times 5 \times 100 + 10 \times 50 \times 100 = 75000$ operations. (!)

The decomposition of the problem in terms of Dynamic Programming (DP) is as follows:

Let $C_{i,j}$ be the minimum cost of multiplying $A_i \times A_{i+1} \times \ldots \times A_j$,

$$C_{i,i} = 0$$

$$C_{i,j} = \min(C_{i,k} + C_{k+1,j} + m_{i-1,k,j}), ~ ~ k \in \{i, \ldots, j-1\}  $$

where $m_{i-1,k,j}$ is the cost of multiplying $(A_iA_{i+1}\ldots A_k)(A_{k+1}\ldots A_j)$

In other words, the minimum possible cost is the minimum between the costs of the various possible choices to split $A_i \times A_{i+1} \times \ldots \times A_j$ into two multiplications.

For example, to calculate the minimum cost of $A_1A_2A_3A_4$, we need to analyze the costs of $A_1(A_2A_3A_4)$, $(A_1A_2)(A_3A_4)$, and $(A_1A_2A_3)A_4$.

In Python, with a bottom-up approach:

In [76]:
from math import inf as oo

def matMult(A):
  n = len(A)
  # cost[i][j] is minimal number of ops to compute A_i*A_{i+1}*...*A_j
  cost = [[+oo for _ in range(n)] for _ in range(n)]

  for i in range(1,n):
    cost[i][i] = 0

  for delta in range(2,n):  # delta, number of matrices multiplied
    for i in range(1,n-delta+1):
      j = i+delta-1
      cost[i][j] = min(cost[i][k] + cost[k+1][j] + A[i-1]*A[k]*A[j]
                       for k in range(i,j))
  return cost[1][-1]

Let's check the first example:

In [77]:
assert matMult([10, 50, 5, 100]) == 7500

This algorithm is $\mathcal{O}(n^3)$



---



Three problem decomposition techniques have been introduced. We can distinguish these techniques by how they deal with the different generated subproblems:

- In divide and conquer, the problem is split into subproblems that do not interact with each other.
  
- In dynamic programming, the problem has optimal substructure and is divided into subproblems that share sub-subproblems.

- In greedy algorithms, the problem has optimal substructure and only requires the answer to the «best» subproblem.

Graphically:

<center><img src='https://raw.githubusercontent.com/jpneto/Prog.I/master/imgs/DQ vs DP vs greedy.png' width=750px></center>

<!--
%%svg

<svg width="800" height="150" xmlns="http://www.w3.org/2000/svg">
 <g>
  <title>Layer 1</title>
  <ellipse ry="10" rx="20" id="svg_1" cy="18.84444" cx="178.10742" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_3" cy="60.49272" cx="128.10742" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_4" cy="60.49272" cx="224.11116" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_5" cy="109.01743" cx="87.11436" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_6" cy="109.01743" cx="149.11116" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_7" cy="109.01743" cx="207.17639" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_8" cy="109.01743" cx="270.18013" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_16" cy="18.84444" cx="604.01743" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_17" cy="60.49273" cx="604.01743" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_18" cy="109.01742" cx="604.01743" stroke="#000" fill="#fff"/>
  <line id="svg_19" y2="50.84444" x2="136.10742" y1="25.84444" x1="164.10742" stroke="#000" fill="none"/>
  <line id="svg_20" y2="50.84444" x2="136.10742" y1="25.84444" x1="164.10742" stroke="#000" fill="none"/>
  <line id="svg_22" y2="50.84444" x2="136.10742" y1="25.84444" x1="164.10742" stroke="#000" fill="none"/>
  <line id="svg_24" y2="50.51852" x2="219.7037" y1="27.55556" x1="191.55555" stroke="#000" fill="none"/>
  <line id="svg_25" y2="97.92593" x2="268.59259" y1="69.03704" x1="235.25925" stroke="#000" fill="none"/>
  <line id="svg_26" y2="97.92593" x2="207.11111" y1="69.77778" x1="218.96296" stroke="#000" fill="none"/>
  <line id="svg_27" y2="98.66667" x2="149.33333" y1="69.77778" x1="136.74074" stroke="#000" fill="none"/>
  <line id="svg_28" y2="98.66667" x2="90.81481" y1="68.2963" x1="113.77778" stroke="#000" fill="none"/>
  <ellipse ry="10" rx="20" id="svg_29" cy="18.84444" cx="425.59567" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_30" cy="60.49272" cx="375.59567" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_31" cy="60.49272" cx="471.59941" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_32" cy="109.01743" cx="334.60262" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_33" cy="109.01743" cx="396.59941" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_34" cy="109.01743" cx="454.66464" stroke="#000" fill="#fff"/>
  <ellipse ry="10" rx="20" id="svg_35" cy="109.01743" cx="517.66838" stroke="#000" fill="#fff"/>
  <line id="svg_36" y2="50.84444" x2="383.59567" y1="25.84444" x1="411.59567" stroke="#000" fill="none"/>
  <line id="svg_37" y2="50.84444" x2="383.59567" y1="25.84444" x1="411.59567" stroke="#000" fill="none"/>
  <line id="svg_38" y2="50.84444" x2="383.59567" y1="25.84444" x1="411.59567" stroke="#000" fill="none"/>
  <line id="svg_39" y2="50.51852" x2="467.19195" y1="27.55556" x1="439.04381" stroke="#000" fill="none"/>
  <line id="svg_40" y2="97.92593" x2="516.08084" y1="69.03704" x1="482.74751" stroke="#000" fill="none"/>
  <line id="svg_41" y2="97.92593" x2="454.59936" y1="69.77778" x1="466.45121" stroke="#000" fill="none"/>
  <line id="svg_42" y2="98.66667" x2="396.82158" y1="69.77778" x1="384.22899" stroke="#000" fill="none"/>
  <line id="svg_43" y2="98.66667" x2="338.30307" y1="68.2963" x1="361.26603" stroke="#000" fill="none"/>
  <line id="svg_44" y2="100.14815" x2="510.8148" y1="64.59259" x1="393.77776" stroke="#000" fill="none"/>
  <line id="svg_45" y2="100.14815" x2="407.1111" y1="68.2963" x1="458.22221" stroke="#000" fill="none"/>
  <line id="svg_46" y2="50.53174" x2="604.14813" y1="29.77778" x1="604.14813" stroke="#000" fill="none"/>
  <line id="svg_47" y2="99.44444" x2="604.14813" y1="70.44444" x1="604.14813" stroke="#000" fill="none"/>
  <text xml:space="preserve" text-anchor="start" font-family="Noto Sans JP" font-size="14" stroke-width="0" id="svg_48" y="139.40741" x="126.42301" stroke="#000" fill="#000000">divide and conquer</text>
  <text xml:space="preserve" text-anchor="start" font-family="Noto Sans JP" font-size="14" stroke-width="0" id="svg_49" y="139.40741" x="373.83939" stroke="#000" fill="#000000">dynamic programming</text>
  <text xml:space="preserve" text-anchor="start" font-family="Noto Sans JP" font-size="14" stroke-width="0" id="svg_50" y="139.40741" x="553.85875" stroke="#000" fill="#000000">greedy algorithm</text>
 </g>

</svg>
-->

Some steps to consider when trying to solve a programming problem P:

+ Is P a well-known problem? Is it a variant of a known problem? Perhaps it is a known problem in desguise... If so, start by studying the solutions found in the literature.

+ If we do some type of fast pre-processing, like sorting or filtering the input data, does it simplify the problem?

+ Is a simple, direct and stupid brute-force approach fast enough? Are you using appropriate data structures?

+ Does a greedy algorithm solves the problem? Can you find some counter-examples? If those exist, what do they say about the problem structure?

+ Can we easily split it into two or more parts, to use a divide and conquer approach? Can the solution exist between the parts? If so, how easy is to combine the results?

+ Ok, nothing else works so far... Can you apply dynamic programming? Try first a top-down approach. Don't forget to cache the results.

+ And what about getting back to brute-force, but using searching techniques, like backtracking?

+ Are you sure there is an efficient way to solve it? Is there a possibility that the problem has exponential complexity? (we have a chapter about this)

+ Can we accept a sub-optimal solution? If so, perhaps some greedy strategies deserve another look (we also have a chapter about this)

## Exercises

<font size="+4" color="blue;green"><b>?</b></font> Define function `missing` that, given a list of non-negative integers in ascending order, returns the smallest element not present in the list. Your implementation should have logarithmic complexity.

<details>
<summary><font color='green'>Hint</font></summary>
Review chapter 14 and use function `search` that generalizes binary search.
</details>

In [ ]:
def missing(xs):
  ...

In [ ]:
assert missing([0,1,2,4,5,6,8]) == 3
assert missing([1,2,4,5,6,8,9]) == 0
assert missing([0,1,2,3,4,5,6]) == 7



---



There is a formula used in fixed-rate loans that defines the monthly installment to be paid given the initial amount, the number of months for payment, and the monthly interest. This formula is implemented in the following function:

In [ ]:
def exactPayment(n_months, total, interest):
  r, n = interest-1, n_months
  return total * (r*(1+r)**n) / ((1+r)**n-1)

For example, if we want to repay 1000 euros over 20 months with 1% interest per month:

In [ ]:
exactPayment(20, 1000.0, 1.01)

55.41531489055138

Let's assume in this exercise that we were not aware of the formula but would like to calculate the monthly installment through approximation.

<font size="+4" color="blue;green"><b>?</b></font> Use the divide and conquer strategy to define function `findPayment` that produces an approximate result to six decimal places of the previous function.

In [ ]:
def findPayment(n_months, total, interest):
  ...



---



<font size="+4" color="blue;green"><b>?</b></font> Solve the next easy puzzle (from Paul Vaderlind's et.al., _The Inquisitive Problem Solver_) with pen and paper. Then solve it using binary search:

> My bathroom scale is set incorrectly but otherwise it works fine. It shows 10 kilograms when Dan stands on it, 14 kilograms when Sarah stands on it, but 22.5 kilograms when Dan and Sarah are both on it. Is the scale set too high?



---



<font size="+4" color="blue;green"><b>?</b></font> You are given a list of integers up to size $1000$. We wish to count how many increasing subsequences are there of length $3$.

For instance, for list `[2,1,5,7,3,4]` there are the following subsequences: `[2,5,7], [2,3,4], [1,5,7], [1,3,4]` so the result should be $4$.

The standard brute force approach has cubic complexity:

In [ ]:
def count_ss3(ns):
  count = 0
  for i,x in enumerate(ns):
    for j,y in enumerate(ns[i+1:], start=i+1):
      for z in ns[j+1:]:
        if x < y < z:
          count += 1
  return count

In [ ]:
assert count_ss3([2,1,5,7,3,4]) == 4

<font size="+4" color="blue;green"><b>?</b></font> Improve this code to achieve quadratic complexity.



---



<font size="+4" color="blue;green"><b>?</b></font> Consider a list `xs` of positive integers. Find all quadruples of elements in `xs` such as `xs[a] * xs[b] * xs[c] == xs[d]` where `a < b < c < d`.

For example, with `xs = [2,1,4,6,8,3,5,10,30,24,12]` these are the solutions:

    ( 2, 1, 4, 8) -->  2 *  1 *  4 ==  8
    ( 2, 1, 6,12) -->  2 *  1 *  6 == 12
    ( 2, 1, 5,10) -->  2 *  1 *  5 == 10
    ( 2, 4, 3,24) -->  2 *  4 *  3 == 24
    ( 2, 3, 5,30) -->  2 *  3 *  5 == 30
    ( 1, 4, 6,24) -->  1 *  4 *  6 == 24
    ( 1, 4, 3,12) -->  1 *  4 *  3 == 12
    ( 1, 6, 5,30) -->  1 *  6 *  5 == 30
    ( 1, 8, 3,24) -->  1 *  8 *  3 == 24
    ( 1, 3,10,30) -->  1 *  3 * 10 == 30

This problem has a standard $\mathcal{O}(n^4)$ solution with four loops:

In [78]:
def triple_mults(xs):
  for a in range(len(xs)):
    for b in range(a+1, len(xs)):
      for c in range(b+1, len(xs)):
        for d in range(c+1, len(xs)):
          if xs[a] * xs[b] * xs[c] == xs[d]:
            yield (xs[a], xs[b], xs[c], xs[d])

In [79]:
xs = [2,1,4,6,8,3,5,10,30,24,12]
print(*triple_mults(xs))

(2, 1, 4, 8) (2, 1, 6, 12) (2, 1, 5, 10) (2, 4, 3, 24) (2, 3, 5, 30) (1, 4, 6, 24) (1, 4, 3, 12) (1, 6, 5, 30) (1, 8, 3, 24) (1, 3, 10, 30)


Implement a $\mathcal{O}(n^2)$ solution inspired in the _meet in the middle_ approach.

In [ ]:
def triple_mults(xs):
  ...



---



<font size="+4" color="blue;green"><b>?</b></font> <font size="+4" color="blue;green"><b>?</b></font> Given a list with $2*n$ integers, split into two partitions each of size $n\lt 16$ to minimize the absolute difference of their sums, and return that difference.



The next program uses backtrack to find the solution:

In [ ]:
def min_diff(ns):

  def solve(i, sum1, len1, sum2, len2):
    """ i: current index
        sum1, sum2: current partition sums
        len1, len2: current partition lengths """
    if len1 == sz//2 and sum1 == half:
      return sum1, sum2+sum(ns[i:])

    if len2 == sz//2 and sum2 == half:
      return sum1+sum(ns[i:]), sum2

    if sz-i == len1-len2:
      return sum1, sum2+sum(ns[i:])

    if sz-i == len2-len1:
      return sum1+sum(ns[i:]), sum2

    sol1_1, sol1_2 = solve(i+1, sum1+ns[i], len1+1, sum2, len2)
    sol2_1, sol2_2 = solve(i+1, sum1, len1, sum2+ns[i], len2+1)

    if abs(sol1_1-sol1_2) < abs(sol2_1-sol2_2):
      return sol1_1, sol1_2
    else:
      return sol2_1, sol2_2

  sz, half = len(ns), sum(ns)//2
  xs, ys = solve(0, 0, 0, 0, 0)
  return abs(xs-ys)

In [ ]:
assert min_diff([0,6,11,17,18,24]) == 6 # [0,11,24] [6,17,18]

However, it is too slow:

In [ ]:
xs = list(range(24))
%timeit min_diff(xs)

3.84 s ± 467 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To speed the calculations use a _meet in the middle_ approach.



---



<center><img src='https://static.mathigon.org/cms/5d23833f6ca757a100c079407ea6694c.png' width=250px></center>

The mathematics of Ancient Egypt only allowed fractions in the form $1/n$.

In this way, other fractions smaller than one, such as $3/4$ or $6/7$, were represented as sums of Egyptian fractions. Any fraction can be represented in this way (in fact, there are several possible solutions). For example,

$$3/4 = 1/2 + 1/4$$

$$6/7 = 1/2 + 1/3 + 1/42$$

This way of representing fractions had some advantages. For example, if it were necessary to distribute three bags of wheat among four workers, how would one do it? In Egyptian notation, the answer is immediate: each worker receives half a bag plus a quarter of a bag.

<font size="+4" color="blue;green"><b>?</b></font> Define function `egyptians`  that takes a fraction (an object of the `Fraction` class) and returns a list of the corresponding Egyptian fractions.

This problem admits a greedy approach.

In [ ]:
from fractions import Fraction

def egyptians(n):
  ...

In [ ]:
assert egyptians(Fraction( 3, 4)) == [Fraction(1, 2), Fraction(1, 4)]
assert egyptians(Fraction( 6, 7)) == [Fraction(1, 2), Fraction(1, 3), Fraction(1, 42)]
assert egyptians(Fraction(61,66)) == [Fraction(1, 2), Fraction(1, 3), Fraction(1, 11)]



---



There are $N$ boxes where we want to store $M$ objects, $1 \leq M \leq 2N$. Each object has a weight $w_i$, and each box can contain up to two objects.

<font size="+4" color="blue;green"><b>?</b></font> Define function `loadBalance` that takes value $N$ and a set of weights $w_1, \ldots, w_M$, and distributes the objects among the boxes to minimize the difference between the box with the heaviest load and the box with the lightest load.

Solve the problem using a greedy strategy.

In [ ]:
def loadBalance(N, ws):
  ...

In [ ]:
print(loadBalance(3, [3,6,8,10]))
print(loadBalance(6, [6,8,2,1,5,3,12,8,14,9]))

[(10, None), (8, None), (6, 3)]
[(14, None), (12, None), (9, 1), (8, 2), (8, 3), (6, 5)]




---



We have $n$ files $f_0, f_1, \ldots, f_{n-1}$ with customer records that we want to merge into a single file (the order of records does not matter). We can merge two files $f_i, f_j$ at a time. Merging them costs $n_i + n_j$, where $n_k$ is the number of records in file $f_k$.

<font size="+4" color="blue;green"><b>?</b></font> Define function `cost` which takes a list of file dimensions and indicates the minimum cost to merge all files into one.

This problem admits a greedy strategy.

In [ ]:
def cost(ns):
  ...

In [ ]:
ns = [500, 120, 400, 1000, 80, 130, 250]

assert cost(ns) == 2480



---



There is a set of $T$ tasks that need to be performed, and each one has a difficulty level $t_i$. Your company has $W$ workers, each with an efficiency level $w_j$. A worker's cost is given by its efficiency. A task can only be performed by a worker who has an efficiency level greater than or equal to its difficulty level. And each worker can only perform a single task.

<font size="+4" color="blue;green"><b>?</b></font> Define the `budget` function that takes a list of task difficulties and a list of worker efficiencies and returns the minimum possible cost to perform all tasks, or `None` if it is impossible to complete the given tasks.

Solve using a greedy strategy.

In [ ]:
def budget(tasks, workers):
  ...

In [ ]:
assert budget([5,4], [7,8,4]) == 11
assert budget([1234567,2345], [12345610,23564,123456,2,3,2344]) == 12369174



---



There will be a vote in $N$ cities with populations $c_1, \ldots, c_N$. It is necessary to distribute $M$ ballot boxes among the cities. Each city must have at least one ballot box. If a city has multiple ballot boxes, its residents are divided equally among all the boxes.

<font size="+4" color="blue;green"><b>?</b></font> Define the `assignBallots` function that proposes a distribution of ballot boxes per city to minimize the maximum number of voters assigned to a ballot box. The function should also return this minimum number of voters per box.

Solve using a greedy strategy.

In [ ]:
def assignBallots(nBallots, cities):
  ...

In [ ]:
assignBallots(12, {'a':500, 'b':2500, 'c':3500, 'd':200})

(625, {'b': 4, 'c': 6, 'a': 1, 'd': 1})



---



At McDonald's, McNugget packages contain $6, 9,$ or $20$ pieces. A number is a _mcnugget_ if it is the sum of a set of McNugget packages. It is known that, starting from $44$, all numbers are _mcnugget_.

<font size="+4" color="blue;green"><b>?</b></font> Define the generator `mcNugget` that generates all numbers that are **not** _mcnugget_.

In [ ]:
def non_mcNugget():
  ...

In [ ]:
assert len(list(non_mcNugget())) == 22

for n in non_mcNugget():
  print(n, end=' ')

1 2 3 4 5 7 8 10 11 13 14 16 17 19 22 23 25 28 31 34 37 43 



---



<font size="+4" color="blue;green"><b>?</b></font> Define function `computeAdditions` that, given two positive integers $N, K$, calculates in how many ways we can sum to $N$ with exactly $K$ non-negative integers.

For example, for $N=3, K=3$, there are ten possibilities: `1+1+1, 1+2+0, 1+0+2, 2+1+0, 2+0+1, 0+1+2, 0+2+1, 3+0+0, 0+3+0, 0+0+3`.

As these results grow too quickly, return the result modulo $10^6$.

In [ ]:
def computeAdditions(N, K):
  ...

In [ ]:
assert computeAdditions(3,3) == 10
assert computeAdditions(63,59) == 649440
assert computeAdditions(33,39) == 100730



---



The problem known as _maximum sum in sequence_ or _range sum query_ (RSQ) refers to the problem of finding, in a list, a sequence of consecutive elements with maximum sum.

For example, given the sequence `23 -10 12 9 -40 32 12 -4 10 6`, the sequence with the maximum sum is `32 12 -4 10 6`, and the total sum is $56$.

<font size="+4" color="blue;green"><b>?</b></font> Define function `RSQ` that, given a list of integers, returns the maximum possible sum of a sequence in that list. If only negative values exist, the function should return zero. Implement the function to be $\mathcal{O}(n)$.

<details>
<summary><font color='green'>Hint</font></summary>
  Investigate Kadane's algorithm.
</details>

In [ ]:
def RSQ(xs):
  ...

In [ ]:
assert RSQ([23,-10,12,9,-40,32,12,-4,10,6]) == 56
assert RSQ([-1,-2,-4,-3]) == 0

Reimplement the function to return the sublist with maximum sum.

In [ ]:
assert RSQ([23,-10,12,9,-40,32,12,-4,10,6]) == [32, 12, -4, 10, 6]



---



<font size="+4" color="blue;green"><b>?</b></font> Define function `subsetsum` that, given a positive integer $n$ and a list of positive integers, returns a pair `(report, list)`. If possible, `list` is a list of elements from the given list whose sum is $n$ (and `report` is `True`). If no solution is possible, the function should return `(False, None)`.

Use dynamic programming in your solution.

In [ ]:
def subsetsum(n, xs):
  ...

In [ ]:
print(subsetsum(12, [3,6,1,5,4]))
print(subsetsum(17, [3,6,1,5,4]))

(True, (4, 5, 3))
(False, ())




---



The _weighted independent set_ (WIS) of a graph is a set of non-adjacent vertices. If each vertex has a value, the maximal WIS is the WIS that has a maximum sum of vertex values. The solution requires an algorithm with exponencial complexity.

Here we consider a path graph, which consists of a sequence of edges $(v_i, v_{i+1})$, just like the next example:

<center><img src='https://raw.githubusercontent.com/jpneto/Prog.I/master/imgs/path_graph_eg.png' width=300px></center>

In this example, the maximal WIS consists of the second and fourth vertices to a total sum of eight.

<font size="+4" color="blue;green"><b>?</b></font> Define function `max_weighted_independent_set` that returns the total sum of its maximal WIS, using dynamic programming.

In [ ]:
def max_weighted_independent_set(ws):
  ...

In [ ]:
assert max_weighted_independent_set([1,4,5,4]) == 8

from random import seed, shuffle
seed(101); xs = list(range(1250)); shuffle(xs)

assert max_weighted_independent_set(xs) == 458853



---



A list is wiggly if the elements at odd indices are greater than their neighboring elements. For example, `[2,3,1,3,1,2]` is a wiggly list.

<font size="+4" color="blue;green"><b>?</b></font> <font size="+4" color="blue;green"><b>?</b></font> Define function `wiggle_sort` that takes a list and returns a wiggly list with the same elements. Assume that the given list can be wiggled. Implement the function in linear time.

In [ ]:
def wiggle_sort(xs):
  ...



---



The Longest Common Subsequence (LCS) is the largest common string between two given strings, if we do not require the characters to be consecutive. For example, given the strings `abcdeyf` and `xaxxbxxcdxxxf`, the LCS is `abcdf`.

Search the internet by the name of the problem and study how this problem can be solved using dynamic programming.

<font size="+4" color="blue;green"><b>?</b></font> <font size="+4" color="blue;green"><b>?</b></font> Define function `lcs` that, given two strings, returns their Longest Common Subsequence.

In [ ]:
def lcs(s1, s2):
  ...

In [ ]:
assert lcs('abcdeyf', 'xaxxbxxcdxxxf') == 'abcdf'



---



A sequence is bitonic if it starts with elements in ascending order and then the remaining elements are in descending order. For example, `[1,2,3,4,3,2,1]` or `[1,4,8,0]` are bitonic.

<font size="+4" color="blue;green"><b>?</b></font> <font size="+4" color="blue;green"><b>?</b></font> Define the function `LBS` that, given a list of elements, returns the longest bitonic subsequence of non-consecutive elements.

This problem is called the Longest Bitonic Subsequence (LBS).

<details>
<summary><font color='green'>Hint</font></summary>
Review the solution for the Longest Common Subsequence. Why not use that algorithm in the first increasing phase and its mirror image in the second decreasing phase?
</details>

In [ ]:
def LBS(xs):
  ...

In [ ]:
print(LBS([0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15, 3]))
print(LBS([4, 2, 5, 9, 7, 6, 10, 3, 1]))

[0, 8, 12, 14, 13, 11, 7, 3]
[4, 5, 9, 7, 6, 3, 1]




---



Consider a grid of size $n \times n$ and a robot placed in one of the cells with coordinates $(x, y)$. The robot moves randomly to one of the four adjacent cells with equal probability. If the robot moves out of the grid, it does not survive.

<font size="+4" color="blue;green"><b>?</b></font> Define the function `survival` that takes the grid dimensions, the coordinates of the initial position, and the number of steps the robot is supposed to take, and returns the probability of survival.

In [ ]:
def survival(n, x, y, steps):
  ...

In [ ]:
from math import isclose

assert isclose(survival( 5,  2,  2,  3), 0.9375)
assert isclose(survival( 7,  3,  3,  3), 1.0)
assert isclose(survival( 1,  0,  0,  3), 0.0)
assert isclose(survival(21, 10, 10, 50), 0.89, rel_tol=1e-3)



---



Now we have placed the robot in the upper-left corner of another grid. This grid has, in some cells, a coin. The robot can move, in each turn, one cell to the right or one cell down. Every time the robot passes through a cell with a coin, it collects it.

<font size="+4" color="blue;green"><b>?</b></font> Define function `collect` that takes a grid and returns the maximum number of coins that can be collected until the robot reaches the bottom-right corner.

In [ ]:
def collect(grid):
  ...

In [ ]:
grid = [[0,0,1,1,0,0],
        [0,1,1,1,0,1],
        [0,0,1,0,0,1],
        [1,0,1,1,1,1],
        [1,1,1,0,1,0]]

assert collect(grid) == 7



---



<font size="+4" color="blue;green"><b>?</b></font> Define function `partition` that takes a list of integers and returns a pair of lists containing all the elements of the original list, and having the same sum. If it is not possible to partition the list, the function should return `None`.

For example, for the list `[3,6,1,5,3]`, the function could return, for example, the pair `[3,6], [1,3,5]`.

In [ ]:
def partition(xs):
  ...

In [ ]:
print(partition([3,6,1,5,3]))
print(partition([7,9,1,3,12,8,3,2,1]))

([3, 6], [1, 3, 5])
([1, 3, 3, 7, 9], [1, 2, 8, 12])




---



<font size="+4" color="blue;green"><b>?</b></font> Define function `levenshtein(word1, word2)` that returns the minimum number of editing operations needed to transform one word into another.

The editing operations can be the following:

+ Insert a letter at any position
    
+ Delete a letter
    
+ Replace one letter with another letter

For example, the distance between `para` and `prol` is three because we need three operations to transform one into the other, that is, `para -> pra -> pro -> prol`.

Your solution should have quadratic complexity, both in time and space, in relation to the number of characters in the given words.

<details>
<summary><font color='green'>Hint</font></summary>
You should represent the information in a matrix where each row corresponds to the letters of the 1st word, and the columns to the letters of the 2nd word. In the given example, the matrix would be initialized like this:

<pre>
        p  r  o  l
  [[ 0  1  2  3  4]
 p [ 1  0  0  0  0]
 a [ 2  0  0  0  0]
 r [ 3  0  0  0  0]
 a [ 4  0  0  0  0]]</pre>
Each position [i][j] in the matrix will represent the distance between the strings word1[:i] and word2[:j]. The final solution after filling the matrix will be in the bottom right cell.

It is possible to fill the matrix starting from the rows above, from left to right. For this example, the matrix, after being filled, will have the following values:
<pre>
        p  r  o  l
  [[ 0  1  2  3  4]
 p [ 1  0  1  2  3]
 a [ 2  1  1  2  3]
 r [ 3  2  1  2  3]
 a [ 4  3  2  2  3]]</pre>
 </details>

In [ ]:
def levenshtein(s1, s2):
  ...

In [ ]:
assert levenshtein('para', 'prol') == 3
assert levenshtein('programming', 'python') == 9